# Grid search vs. genetic algorithm for hyperparameter tuning

Comparing two hyperparameter tuning methods: grid search and a genetic algorithm (implemented in the `gentun` library).

For this experiment, I tuned the following parameters of a gradient boosting model (`sklearn.ensemble.HistGradientBoostingRegressor`):
- `learning_rate`: selected on a log scale between 0.01 and 1
- `max_leaf_nodes`: one of 4, 8, 16, 32, ..., 256, None (no limit)
- `max_depth`: one of 3, 4, 5, ..., 10, None (no limit)
- `min_samples_leaf`: one of 1, 2, 3, ..., 50
- `max_features`: selected between 0.25 and 1

For the genetic algorithm, during mutations `learning_rate` and `max_features` were sampled uniformly on the given range and the other parameters were selected at random from the given list of values. For the grid search, each parameter had 4 possible values evenly spaced within the available range. (For `max_leaf_nodes` and `max_depth`, None was considered the last element of the range and was included in the grid.)

Each model was evaluated using stratified 5-fold cross-validation. For more details on the stratified k-fold algorithm for regression, see [`cv/visualize_methods.ipynb`](cv/visualize_methods.ipynb).

The search itself is implemented in [`search.py`](search.py), this notebook just analyzes the results.

In [1]:
import pickle

import pandas as pd
import sklearn.ensemble

import preprocessing
from search import GradientBoostingHandler  # required for pickle loading to work

In [2]:
X_train, y_train = preprocessing.read_tsv_with_all_features("../data/tournament_dataset/train.tsv")
X_test, y_test = preprocessing.read_tsv_with_all_features("../data/tournament_dataset/test.tsv")

In [3]:
def get_top_individuals(path):
    with open(path, "rb") as fin:
        population = pickle.load(fin)
    rows = [ind.hyperparameters | {"fitness": ind.fitness} for ind in population]
    return pd.DataFrame(rows).sort_values("fitness", ascending=False)

The best hyperparameter combinations found by the grid search:

In [4]:
grid_results = get_top_individuals("results/grid_population.p")
grid_results.head(10)

,learning_rate,max_leaf_nodes,max_depth,min_samples_leaf,max_features,fitness
738,0.215443,NaN,8.0,1,0.75,0.974908
722,0.215443,NaN,5.0,1,0.75,0.973959
691,0.215443,128.0,NaN,1,1.00,0.973555
675,0.215443,128.0,8.0,1,1.00,0.971692
672,0.215443,128.0,8.0,1,0.25,0.970350
658,0.215443,128.0,5.0,1,0.75,0.969136
721,0.215443,NaN,5.0,1,0.50,0.967977
498,0.046416,NaN,NaN,1,0.75,0.967962
673,0.215443,128.0,8.0,1,0.50,0.966666
497,0.046416,NaN,NaN,1,0.50,0.965710


The best hyperparameter combinations found by the genetic algorithm (in the final population):

In [5]:
gen_results = get_top_individuals("results/genetic_population.p")
gen_results.head(10)

,learning_rate,max_leaf_nodes,max_depth,min_samples_leaf,max_features,fitness
0,0.362595,None,9.0,1,0.909082,0.975999
2,0.362595,None,9.0,1,0.909082,0.975999
14,0.362595,None,9.0,1,0.909082,0.975999
4,0.362595,None,9.0,1,0.909082,0.975999
7,0.362595,None,9.0,1,0.909082,0.975999
6,0.362595,None,9.0,1,0.909082,0.975999
8,0.362595,None,9.0,1,0.909082,0.975999
9,0.362595,None,9.0,1,0.909082,0.975999
11,0.362595,None,9.0,1,0.909082,0.975999
10,0.362595,None,9.0,1,0.909082,0.975999


A lot of these are identical, so here's just the unique ones:

In [6]:
gen_results.drop_duplicates(subset=gen_results.columns[:-1])

,learning_rate,max_leaf_nodes,max_depth,min_samples_leaf,max_features,fitness
0,0.362595,None,9.0,1,0.909082,0.975999
5,0.362595,None,9.0,1,0.893395,0.973801
3,0.362595,None,9.0,1,0.986228,0.967131
47,0.362595,None,NaN,1,0.909082,0.964684
40,0.044916,None,9.0,1,0.909082,0.959837
23,0.104592,None,9.0,1,0.909082,0.959653
39,0.362595,None,7.0,1,0.909082,0.944679
38,0.010114,None,9.0,1,0.909082,0.511301


It looks like the best combination found by the generic algorithm is slightly better than the one found by the grid search.

Let's try both of these combinations to train a model on the entire training set and see which one is better on the test set.

In [7]:
for results, name in [(grid_results, "grid search"), (gen_results, "genetic algorithm")]:
    kwargs = results.iloc[0].to_dict()
    del kwargs["fitness"]
    for key in ["max_leaf_nodes", "max_depth", "min_samples_leaf"]:
        kwargs[key] = int(kwargs[key]) if not pd.isna(kwargs[key]) else None

    print(f"{name} best combination:")
    print(f"  {kwargs}")

    model = preprocessing.make_pipeline(
        sklearn.ensemble.HistGradientBoostingRegressor(
            categorical_features=preprocessing.CATEGORICAL_FEATURES, random_state=27, **kwargs
        ),
        ohe=False
    )
    print(f"  test set score = {model.fit(X_train, y_train).score(X_test, y_test):.3f}\n")

grid search best combination:
  {'learning_rate': 0.21544346901036987, 'max_leaf_nodes': None, 'max_depth': 8, 'min_samples_leaf': 1, 'max_features': 0.75}
  test set score = 0.956

genetic algorithm best combination:
  {'learning_rate': 0.36259501134643735, 'max_leaf_nodes': None, 'max_depth': 9, 'min_samples_leaf': 1, 'max_features': 0.909082085917459}
  test set score = 0.969



The genetic algorithm has the better test set score.

Despite the score difference, the best hyperparameter combinations found by both algorithms are fairly similar. It appears that the genetic algorithm was able to find a better combination of hyperparameters because it had more freedom to select hyperparameter values from within the available range. For example, for the `max_features` hyperparameter the grid search had only 4 possible options: 0.25, 0.5, 0.75 and 1. (Of course, the number of options could be increased, but this would significantly hurt performance: in this case adding another possible value of `max_features` would slow down the algorithm by 25\%.) The genetic algorithm was able to find the more optimal value of ~0.91, since during mutations new values of `max_features` are sampled uniformly from the [0.25, 1] range. However, this does depend on luck, and it's possible that in another experiment the genetic algorithm would never generate that mutation and end up with a worse result instead. The grid search at least guarantees that every combination of hyperparameter values in the grid will be tested, even if those values aren't optimal.

Here's how much time each algorithm took to perform the search (taken from the `gentun` training logs):
- grid search (4 possible values for each of 5 hyperparameters, 1024 combinations evaluated): 2 hours 14 minutes
- genetic algorithm (20 generations, population size 50, 1030 combinations evaluated): 1 hour 59 minutes

The genetic algorithm seems to be slightly faster, even though the number of trained models is about the same. One reason for this could be that the grid search is forced to evaluate every single parameter combination, including those which are slow to train. If these slow combinations also happen to produce worse model quality (e.g. learning rate too small), then they will quickly "go extinct" in the generic algorithm and it won't spend as much time evaluating them.

It should be noted that the genetic algorithm could be made even faster with caching. Since there are usually many identical individuals (hyperparameter combinations) in the population, the genetic algorithm often has to train and evaluate the model with the exact same hyperparameters several times. If the scores for these models were cached, the algorithm would be much more efficient. The `gentun` library does not seem to implement this natively, but it could be implemented in a custom `Handler` subclass.

In conclusion, here's how the two methods compare on different factors:

- **Quality:** the combination found by the genetic algorithm produces a better score both when cross-validating and on the test set.
- **Performance:** the genetic algorithm is faster, likely because it does not have to evaluate many slow low-quality hyperparameter combinations. However, this could change on a different task where the slower models actually have higher quality. In addition, the genetic algorithm can be made even faster with caching, unlike grid search.
- **Comparison of selected hyperparameters:** they ended up being fairly similar, but the genetic algorithm benefitted from being able to select values that don't appear in the grid search grid.
- **Influence of random chance:** the genetic algorithm could produce different results depending on the random seed, whereas grid search is entirely deterministic. More experiments would be needed to check how much the genetic algorithm's quality depends on the randomness of the mutations.

The model trained with the best hyperparameters found by the genetic algorithm can be found in [`models/hist_gradient_boosting_tuned.p`](../models/hist_gradient_boosting_tuned.p). The training script for that model is [`train_tuned.py`](train_tuned.py).